In [1]:
import tensorflow as tf

In [10]:
#declare the list of feature
feature_columns=[tf.feature_column.numeric_column("x",shape=[1])]

In [11]:
#creating estimators
regressor=tf.estimator.LinearRegressor(feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_tf_random_seed': 1, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_model_dir': 'C:\\Users\\SREENATH\\AppData\\Local\\Temp\\tmp9kvdgflt', '_keep_checkpoint_every_n_hours': 10000, '_session_config': None}


In [12]:
import numpy as np
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

In [13]:
#reading the data from numpy data
input_fn=tf.estimator.inputs.numpy_input_fn({"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)

In [14]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)


In [15]:
regressor.train(input_fn=input_fn,steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\SREENATH\AppData\Local\Temp\tmp9kvdgflt\model.ckpt.
INFO:tensorflow:step = 1, loss = 14.0
INFO:tensorflow:global_step/sec: 799.552
INFO:tensorflow:step = 101, loss = 0.205698 (0.141 sec)
INFO:tensorflow:global_step/sec: 740.449
INFO:tensorflow:step = 201, loss = 0.056755 (0.119 sec)
INFO:tensorflow:global_step/sec: 797.489
INFO:tensorflow:step = 301, loss = 0.00472396 (0.125 sec)
INFO:tensorflow:global_step/sec: 860.69
INFO:tensorflow:step = 401, loss = 0.00102679 (0.116 sec)
INFO:tensorflow:global_step/sec: 794.844
INFO:tensorflow:step = 501, loss = 0.000377659 (0.126 sec)
INFO:tensorflow:global_step/sec: 853.693
INFO:tensorflow:step = 601, loss = 2.73171e-05 (0.117 sec)
INFO:tensorflow:global_step/sec: 804.269
INFO:tensorflow:step = 701, loss = 5.92962e-06 (0.124 sec)
INFO:tensorflow:global_step/sec: 751.577
INFO:tensorflow:step = 801, loss = 1.6647e-06 (0.135 sec)
INFO:tensorflow:globa

In [17]:
# Here we evaluate how well our model did.
train_metrics = regressor.evaluate(input_fn=train_input_fn)
print("train metrics: %r"% train_metrics)

INFO:tensorflow:Starting evaluation at 2017-12-17-05:54:10
INFO:tensorflow:Restoring parameters from C:\Users\SREENATH\AppData\Local\Temp\tmp9kvdgflt\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-12-17-05:54:11
INFO:tensorflow:Saving dict for global step 1000: average_loss = 1.02939e-08, global_step = 1000, loss = 4.11757e-08
train metrics: {'loss': 4.1175664e-08, 'global_step': 1000, 'average_loss': 1.0293916e-08}


In [18]:
eval_metrics = regressor.evaluate(input_fn=eval_input_fn)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Starting evaluation at 2017-12-17-05:54:41
INFO:tensorflow:Restoring parameters from C:\Users\SREENATH\AppData\Local\Temp\tmp9kvdgflt\model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-12-17-05:54:43
INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.00253344, global_step = 1000, loss = 0.0101338
eval metrics: {'loss': 0.010133768, 'global_step': 1000, 'average_loss': 0.0025334419}


# In custom estimators we are building the model function ourself only

In [29]:
def model_fun(features,labels,mode):
    # Build a linear model and predict values
    W = tf.get_variable("W", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    y = W*features['x'] + b
    # Loss sub-graph
    loss = tf.reduce_sum(tf.square(y - labels))
    # Training sub-graph
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
    # EstimatorSpec connects subgraphs we built to the
    # appropriate functionality.
    return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

In [31]:
estimator = tf.estimator.Estimator(model_fn=model_fun)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_tf_random_seed': 1, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_log_step_count_steps': 100, '_model_dir': 'C:\\Users\\SREENATH\\AppData\\Local\\Temp\\tmpaf3d2i8_', '_keep_checkpoint_every_n_hours': 10000, '_session_config': None}


In [32]:
# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\SREENATH\AppData\Local\Temp\tmpaf3d2i8_\model.ckpt.
INFO:tensorflow:step = 1, loss = 16.3362220696
INFO:tensorflow:global_step/sec: 682.228
INFO:tensorflow:step = 101, loss = 0.042222369076 (0.148 sec)
INFO:tensorflow:global_step/sec: 854.089
INFO:tensorflow:step = 201, loss = 0.0057350896855 (0.117 sec)
INFO:tensorflow:global_step/sec: 825.86
INFO:tensorflow:step = 301, loss = 0.000167320064898 (0.121 sec)
INFO:tensorflow:global_step/sec: 825.855
INFO:tensorflow:step = 401, loss = 5.27947602925e-06 (0.120 sec)
INFO:tensorflow:global_step/sec: 799.428
INFO:tensorflow:step = 501, loss = 2.47174680156e-06 (0.126 sec)
INFO:tensorflow:global_step/sec: 799.431
INFO:tensorflow:step = 601, loss = 7.87427250829e-08 (0.125 sec)
INFO:tensorflow:global_step/sec: 825.847
INFO:tensorflow:step = 701, loss = 2.15164175898e-08 (0.121 sec)
INFO:tensorflow:global_step/sec: 571.025
INFO:tensorflow:step = 80